This function `extract_save` can extract data from the XML file used to backup messages. It extracts:

1. Body of the message in English or Hindi
2. Date stamp
3. Date Sent
4. Service Center Number
5. Human-readable Date & Time
6. Message Title

In [1]:
import xml.etree.ElementTree as ET
import pandas as pd
import csv

def _extract_sms_data(xml_file_path):
    # Parse the XML file with explicit UTF-8 encoding
    try:
        with open(xml_file_path, encoding='utf-8') as file:
            tree = ET.parse(file)
            root = tree.getroot()
    except FileNotFoundError:
        print(f"Error: File {xml_file_path} not found.")
        return None
    except ET.ParseError:
        print("Error: Invalid XML format.")
        return None
    except UnicodeDecodeError:
        print("Error: Unable to decode file with UTF-8 encoding.")
        return None

    # List to store extracted data
    data = []

    # Define the properties to extract
    properties = ['address', 'body', 'readable_date', 'service_center', 'date', 'date_sent']

    # Iterate through each <sms> element
    for sms in root.findall('sms'):
        # Extract specified attributes, use None if attribute is missing
        sms_data = {prop: sms.get(prop, None) for prop in properties}
        data.append(sms_data)

    # Create DataFrame
    df = pd.DataFrame(data, columns=properties)
    return df

def extract_save(xml_file_path, save_path='sms_data.csv'):
    df = _extract_sms_data(xml_file_path)
    print(df)
    if df is not None:
        # Save DataFrame to CSV with UTF-8 encoding to preserve Hindi text
        df.to_csv(save_path, index=False, encoding='utf-8-sig')
        print(f"\nDataFrame saved to {save_path} with UTF-8 encoding")

In [2]:
# extract_save('sms-20250722210237.xml','one.csv')
# extract_save('sms-20250724144910.xml','two.csv')

In [3]:
import pandas as pd
df = _extract_sms_data('sms-20250722210237.xml')
df.head()

,address,body,readable_date,service_center,date,date_sent
0,JM-PAYZAP-S,Login attempt on your PayZapp account at 06:52...,15 May 2025 7:32:18 pm,+917021075036,1747317738643,1747317736000
1,51501,971115 Message ID: 0PYGGQiwfmU,15 May 2025 7:48:36 pm,+918299901123,1747318716822,1747318715000
2,VM-HDFCBK-S,Sent Rs.420.00\nFrom HDFC Bank A/C *9442\nTo S...,15 May 2025 8:51:33 pm,+919823000122,1747322493534,1747322491000
3,AE-AIRTEL-P,फ्री हैलोट्यून आपका इंतज़ार कर रही है!\nआज ही ए...,16 May 2025 12:37:35 pm,+919831029607,1747379255850,1747379253000
4,AE-AIRSLF,"नमस्ते, अपना करेंट पैक डिटेल घर बैठे एयरटेल थै...",16 May 2025 2:23:40 pm,+919840011991,1747385620009,1747385618000


In [4]:
!pip install langdetect pandas

In [5]:
import re
def clean_message(text):
    text = str(text)
    print('\n',text,'\n')
    text = re.sub(r'\d{4,}', ' [NUM] ', text) # replace long numbers
    # print(text)
    text = re.sub(r'http\S+', ' [URL] ', text) # replace URLs
    # print(text)
    text = re.sub(r'[^\w\s\u0900-\u097F]', '', text) # Remove special char except hindi
    # print(text)
    text = re.sub(r'\s+', ' ', text) # normalize spaces
    print('\n',text,'\n')
    return text.strip()

from langdetect import detect
def detect_language(text):
    try:
        lang = detect(text)
        return 'en' if lang!='hi' else lang # ex. 'en', 'hi'
    except:
        return 'unknown'

# print(df['body'].iloc[1])

df['cleaned_message'] = df['body'].apply(clean_message)
df['language'] = df['body'].apply(detect_language)
df.head()

    


 Login attempt on your PayZapp account at 06:52 PM.
If it wasn't you, contact customer support: https://hdfcbk.io/HDFCBK/s/B5LJXVBd 


 Login attempt on your PayZapp account at 0652 PM If it wasnt you contact customer support URL  


 971115 Message ID: 0PYGGQiwfmU 


  NUM Message ID 0PYGGQiwfmU 


 Sent Rs.420.00
From HDFC Bank A/C *9442
To SUNY
On 15/05/25
Ref 513576240499
Not You?
Call 18002586161/SMS BLOCK UPI to 7308080808
 


 Sent Rs42000 From HDFC Bank AC NUM To SUNY On 150525 Ref NUM Not You Call NUM SMS BLOCK UPI to NUM  


 फ्री हैलोट्यून आपका इंतज़ार कर रही है!
आज ही एयरटेल थैंक्स ऐप पर सेट करें| i.airtel.in/Free_hellotune 


 फ्री हैलोट्यून आपका इंतज़ार कर रही है आज ही एयरटेल थैंक्स ऐप पर सेट करें iairtelinFree_hellotune 


 नमस्ते, अपना करेंट पैक डिटेल घर बैठे एयरटेल थैंक्स ऐप पर देखें। https://i.airtel.in/prc लिंक पर क्लिक करें 


 नमस्ते अपना करेंट पैक डिटेल घर बैठे एयरटेल थैंक्स ऐप पर देखें। URL लिंक पर क्लिक करें 


 A/c *9456 Debited for Rs:10.00 on 16-05-2025 18:08:

,address,body,readable_date,service_center,date,date_sent,cleaned_message,language
0,JM-PAYZAP-S,Login attempt on your PayZapp account at 06:52...,15 May 2025 7:32:18 pm,+917021075036,1747317738643,1747317736000,Login attempt on your PayZapp account at 0652 ...,en
1,51501,971115 Message ID: 0PYGGQiwfmU,15 May 2025 7:48:36 pm,+918299901123,1747318716822,1747318715000,NUM Message ID 0PYGGQiwfmU,en
2,VM-HDFCBK-S,Sent Rs.420.00\nFrom HDFC Bank A/C *9442\nTo S...,15 May 2025 8:51:33 pm,+919823000122,1747322493534,1747322491000,Sent Rs42000 From HDFC Bank AC NUM To SUNY On ...,en
3,AE-AIRTEL-P,फ्री हैलोट्यून आपका इंतज़ार कर रही है!\nआज ही ए...,16 May 2025 12:37:35 pm,+919831029607,1747379255850,1747379253000,फ्री हैलोट्यून आपका इंतज़ार कर रही है आज ही एयर...,hi
4,AE-AIRSLF,"नमस्ते, अपना करेंट पैक डिटेल घर बैठे एयरटेल थै...",16 May 2025 2:23:40 pm,+919840011991,1747385620009,1747385618000,नमस्ते अपना करेंट पैक डिटेल घर बैठे एयरटेल थैं...,hi


In [6]:
!pip install nltk indic-nlp-library

In [7]:
print({x: df[df['language'] == x].shape[0] for x in df['language'].unique()})
df[df['language'] == 'hi']

{'en': 275, 'hi': 22}


,address,body,readable_date,service_center,date,date_sent,cleaned_message,language
3,AE-AIRTEL-P,फ्री हैलोट्यून आपका इंतज़ार कर रही है!\nआज ही ए...,16 May 2025 12:37:35 pm,+919831029607,1747379255850,1747379253000,फ्री हैलोट्यून आपका इंतज़ार कर रही है आज ही एयर...,hi
4,AE-AIRSLF,"नमस्ते, अपना करेंट पैक डिटेल घर बैठे एयरटेल थै...",16 May 2025 2:23:40 pm,+919840011991,1747385620009,1747385618000,नमस्ते अपना करेंट पैक डिटेल घर बैठे एयरटेल थैं...,hi
24,AD-TRAIND-G,स्पैम कॉल से परेशान हैं? रिपोर्ट करें और स्पैम...,19 May 2025 1:57:14 am,+919831029344,1747600034889,1747600014000,स्पैम कॉल से परेशान हैं रिपोर्ट करें और स्पैम ...,hi
31,56321,अब बेफिक्र होकर डाटा का इस्तेमाल करें क्योंकि ...,19 May 2025 1:22:48 pm,+919810051688,1747641168501,1747641157000,अब बेफिक्र होकर डाटा का इस्तेमाल करें क्योंकि ...,hi
32,AD-AIRTEL-S,यदि आपके पास केवाईसी दस्तावेजों/यूजर आईडी/पासव...,19 May 2025 1:34:19 pm,+919831029607,1747641859078,1747641855000,यदि आपके पास केवाईसी दस्तावेजोंयूजर आईडीपासवर्...,hi
64,56321,अब बेफिक्र होकर डाटा का इस्तेमाल करें क्योंकि ...,24 May 2025 4:55:00 pm,+919810051289,1748085900262,1748085898000,अब बेफिक्र होकर डाटा का इस्तेमाल करें क्योंकि ...,hi
67,AD-AIRTEL-S,एयरटेल अब बन गया है सुरक्षित नेटवर्क।\nहमारी फ...,24 May 2025 6:55:50 pm,+919831029607,1748093150704,1748093148000,एयरटेल अब बन गया है सुरक्षित नेटवर्क। हमारी फ़...,hi
91,AE-650024-P,बिना किसी अतिरिक्त शुल्क के पाएँ अनलिमिटेड 5G ...,28 May 2025 1:52:53 pm,+919840011981,1748420573228,1748420571000,बिना किसी अतिरिक्त शुल्क के पाएँ अनलिमिटेड 5G ...,hi
102,AE-650024-P,Airtel Xstream Play पर बिना किसी अतिरिक्त शुल्...,29 May 2025 1:24:50 pm,+919840011981,1748505290397,1748505289000,Airtel Xstream Play पर बिना किसी अतिरिक्त शुल्...,hi
114,AE-650024-P,बिना किसी अतिरिक्त शुल्क के पाएँ अनलिमिटेड 5G ...,1 Jun 2025 12:43:40 pm,+919831029607,1748762020318,1748762018000,बिना किसी अतिरिक्त शुल्क के पाएँ अनलिमिटेड 5G ...,hi


In [8]:
!pip install indic-nlp-library

In [10]:
# Removing stop words
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')
eng_stop = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\mhsuh\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [11]:
hinglish_stop = set({
    'aadi', 'aaj', 'aap', 'aapne', 'aata', 'aati', 'aaya', 'aaye', 'ababbe', 'abbey', 'abe', 'abhi',
    'able', 'about', 'above', 'accha', 'according', 'accordingly', 'acha', 'achcha', 'across',
    'actually', 'after', 'afterwards', 'again', 'against', 'agar', 'ain', 'aint', "ain't", 'aisa',
    'aise', 'aisi', 'alag', 'all', 'allow', 'allows', 'almost', 'alone', 'along', 'already', 'also',
    'although', 'always', 'am', 'among', 'amongst', 'an', 'and', 'andar', 'another', 'any', 'anybody',
    'anyhow', 'anyone', 'anything', 'anyway', 'anyways', 'anywhere', 'ap', 'apan', 'apart', 'apna',
    'apnaa', 'apne', 'apni', 'appear', 'are', 'aren', 'arent', "aren't", 'around', 'arre', 'as',
    'aside', 'ask', 'asking', 'at', 'aur', 'avum', 'aya', 'aye', 'baad', 'baar', 'bad', 'bahut',
    'bana', 'banae', 'banai', 'banao', 'banaya', 'banaye', 'banayi', 'banda', 'bande', 'bandi', 'bane',
    'bani', 'bas', 'bata', 'batao', 'bc', 'be', 'became', 'because', 'become', 'becomes', 'becoming',
    'been', 'before', 'beforehand', 'behind', 'being', 'below', 'beside', 'besides', 'best', 'better',
    'between', 'beyond', 'bhai', 'bheetar', 'bhi', 'bhitar', 'bht', 'bilkul', 'bohot', 'bol', 'bola',
    'bole', 'boli', 'bolo', 'bolta', 'bolte', 'bolti', 'both', 'brief', 'bro', 'btw', 'but', 'by',
    'came', 'can', 'cannot', 'cant', "can't", 'cause', 'causes', 'certain', 'certainly', 'chahiye',
    'chaiye', 'chal', 'chalega', 'chhaiye', 'clearly', "c'mon", 'com', 'come', 'comes', 'could',
    'couldn', 'couldnt', "couldn't", 'd', 'de', 'dede', 'dega', 'degi', 'dekh', 'dekha', 'dekhe',
    'dekhi', 'dekho', 'denge', 'dhang', 'di', 'did', 'didn', 'didnt', "didn't", 'dijiye', 'diya',
    'diyaa', 'diye', 'diyo', 'do', 'does', 'doesn', 'doesnt', "doesn't", 'doing', 'done', 'dono',
    'dont', "don't", 'doosra', 'doosre', 'down', 'downwards', 'dude', 'dunga', 'dungi', 'during',
    'dusra', 'dusre', 'dusri', 'dvaara', 'dvara', 'dwaara', 'dwara', 'each', 'edu', 'eg', 'eight',
    'either', 'ek', 'else', 'elsewhere', 'enough', 'etc', 'even', 'ever', 'every', 'everybody',
    'everyone', 'everything', 'everywhere', 'ex', 'exactly', 'example', 'except', 'far', 'few',
    'fifth', 'fir', 'first', 'five', 'followed', 'following', 'follows', 'for', 'forth', 'four',
    'from', 'further', 'furthermore', 'gaya', 'gaye', 'gayi', 'get', 'gets', 'getting', 'ghar',
    'given', 'gives', 'go', 'goes', 'going', 'gone', 'good', 'got', 'gotten', 'greetings', 'guys',
    'haan', 'had', 'hadd', 'hadn', 'hadnt', "hadn't", 'hai', 'hain', 'hamara', 'hamare', 'hamari',
    'hamne', 'han', 'happens', 'har', 'hardly', 'has', 'hasn', 'hasnt', "hasn't", 'have', 'haven',
    'havent', "haven't", 'having', 'he', 'hello', 'help', 'hence', 'her', 'here', 'hereafter',
    'hereby', 'herein', "here's", 'hereupon', 'hers', 'herself', "he's", 'hi', 'him', 'himself',
    'his', 'hither', 'hm', 'hmm', 'ho', 'hoga', 'hoge', 'hogi', 'hona', 'honaa', 'hone', 'honge',
    'hongi', 'honi', 'hopefully', 'hota', 'hotaa', 'hote', 'hoti', 'how', 'howbeit', 'however',
    'hoyenge', 'hoyengi', 'hu', 'hua', 'hue', 'huh', 'hui', 'hum', 'humein', 'humne', 'hun', 'huye',
    'huyi', 'i', "i'd", 'idk', 'ie', 'if', "i'll", "i'm", 'imo', 'in', 'inasmuch', 'inc', 'inhe',
    'inhi', 'inho', 'inka', 'inkaa', 'inke', 'inki', 'inn', 'inner', 'inse', 'insofar', 'into',
    'inward', 'is', 'ise', 'isi', 'iska', 'iskaa', 'iske', 'iski', 'isme', 'isn', 'isne', 'isnt',
    "isn't", 'iss', 'isse', 'issi', 'isski', 'it', "it'd", "it'll", 'itna', 'itne', 'itni', 'itno',
    'its', "it's", 'itself', 'ityaadi', 'ityadi', "i've", 'ja', 'jaa', 'jab', 'jabh', 'jaha',
    'jahaan', 'jahan', 'jaisa', 'jaise', 'jaisi', 'jata', 'jayega', 'jidhar', 'jin', 'jinhe', 'jinhi',
    'jinho', 'jinhone', 'jinka', 'jinke', 'jinki', 'jinn', 'jis', 'jise', 'jiska', 'jiske', 'jiski',
    'jisme', 'jiss', 'jisse', 'jitna', 'jitne', 'jitni', 'jo', 'just', 'jyaada', 'jyada', 'k', 'ka',
    'kaafi', 'kab', 'kabhi', 'kafi', 'kaha', 'kahaa', 'kahaan', 'kahan', 'kahi', 'kahin', 'kahte',
    'kaisa', 'kaise', 'kaisi', 'kal', 'kam', 'kar', 'kara', 'kare', 'karega', 'karegi', 'karen',
    'karenge', 'kari', 'karke', 'karna', 'karne', 'karni', 'karo', 'karta', 'karte', 'karti', 'karu',
    'karun', 'karunga', 'karungi', 'kaun', 'kaunsa', 'kayi', 'kch', 'ke', 'keep', 'keeps', 'keh',
    'kehte', 'kept', 'khud', 'ki', 'kin', 'kine', 'kinhe', 'kinho', 'kinka', 'kinke', 'kinki', 'kinko',
    'kinn', 'kino', 'kis', 'kise', 'kisi', 'kiska', 'kiske', 'kiski', 'kisko', 'kisliye', 'kisne',
    'kitna', 'kitne', 'kitni', 'kitno', 'kiya', 'kiye', 'know', 'known', 'knows', 'ko', 'koi', 'kon',
    'konsa', 'koyi', 'krna', 'krne', 'kuch', 'kuchch', 'kuchh', 'kul', 'kull', 'kya', 'kyaa', 'kyu',
    'kyuki', 'kyun', 'kyunki', 'lagta', 'lagte', 'lagti', 'last', 'lately', 'later', 'le', 'least',
    'lekar', 'lekin', 'less', 'lest', 'let', "let's", 'li', 'like', 'liked', 'likely', 'little',
    'liya', 'liye', 'll', 'lo', 'log', 'logon', 'lol', 'look', 'looking', 'looks', 'ltd', 'lunga',
    'm', 'maan', 'maana', 'maane', 'maani', 'maano', 'magar', 'mai', 'main', 'maine', 'mainly',
    'mana', 'mane', 'mani', 'mano', 'many', 'mat', 'may', 'maybe', 'me', 'mean', 'meanwhile', 'mein',
    'mera', 'mere', 'merely', 'meri', 'might', 'mightn', 'mightnt', "mightn't", 'mil', 'mjhe',
    'more', 'moreover', 'most', 'mostly', 'much', 'mujhe', 'must', 'mustn', 'mustnt', "mustn't",
    'my', 'myself', 'na', 'naa', 'naah', 'nahi', 'nahin', 'nai', 'name', 'namely', 'nd', 'ne', 'near',
    'nearly', 'necessary', 'neeche', 'need', 'needn', 'neednt', "needn't", 'needs', 'neither',
    'never', 'nevertheless', 'new', 'next', 'nhi', 'nine', 'no', 'nobody', 'non', 'none', 'noone',
    'nope', 'nor', 'normally', 'not', 'nothing', 'novel', 'now', 'nowhere', 'o', 'obviously', 'of',
    'off', 'often', 'oh', 'ok', 'okay', 'old', 'on', 'once', 'one', 'ones', 'only', 'onto', 'or',
    'other', 'others', 'otherwise', 'ought', 'our', 'ours', 'ourselves', 'out', 'outside', 'over',
    'overall', 'own', 'par', 'pata', 'pe', 'pehla', 'pehle', 'pehli', 'people', 'per', 'perhaps',
    'phla', 'phle', 'phli', 'placed', 'please', 'plus', 'poora', 'poori', 'provides', 'pura', 'puri',
    'q', 'que', 'quite', 'raha', 'rahaa', 'rahe', 'rahi', 'rakh', 'rakha', 'rakhe', 'rakhen', 'rakhi',
    'rakho', 'rather', 're', 'really', 'reasonably', 'regarding', 'regardless', 'regards', 'rehte',
    'rha', 'rhaa', 'rhe', 'rhi', 'ri', 'right', 's', 'sa', 'saara', 'saare', 'saath', 'sab', 'sabhi',
    'sabse', 'sahi', 'said', 'sakta', 'saktaa', 'sakte', 'sakti', 'same', 'sang', 'sara', 'sath',
    'saw', 'say', 'saying', 'says', 'se', 'second', 'secondly', 'see', 'seeing', 'seem', 'seemed',
    'seeming', 'seems', 'seen', 'self', 'selves', 'sensible', 'sent', 'serious', 'seriously', 'seven',
    'several', 'shall', 'shan', 'shant', "shan't", 'she', "she's", 'should', 'shouldn', 'shouldnt',
    "shouldn't", "should've", 'si', 'sir', 'sir.', 'since', 'six', 'so', 'soch', 'some', 'somebody',
    'somehow', 'someone', 'something', 'sometime', 'sometimes', 'somewhat', 'somewhere', 'soon',
    'still', 'sub', 'such', 'sup', 'sure', 't', 'tab', 'tabh', 'tak', 'take', 'taken', 'tarah',
    'teen', 'teeno', 'teesra', 'teesre', 'teesri', 'tell', 'tends', 'tera', 'tere', 'teri', 'th',
    'tha', 'than', 'thank', 'thanks', 'thanx', 'that', "that'll", 'thats', "that's", 'the', 'theek',
    'their', 'theirs', 'them', 'themselves', 'then', 'thence', 'there', 'thereafter', 'thereby',
    'therefore', 'therein', "there's", 'thereupon', 'these', 'they', "they'd", "they'll", "they're",
    "they've", 'thi', 'thik', 'thing', 'think', 'thinking', 'third', 'this', 'tho', 'thoda', 'thodi',
    'thorough', 'thoroughly', 'those', 'though', 'thought', 'three', 'through', 'throughout', 'thru',
    'thus', 'tjhe', 'to', 'together', 'toh', 'too', 'took', 'toward', 'towards', 'tried', 'tries',
    'true', 'truly', 'try', 'trying', 'tu', 'tujhe', 'tum', 'tumhara', 'tumhare', 'tumhari', 'tune',
    'twice', 'two', 'um', 'umm', 'un', 'under', 'unhe', 'unhi', 'unho', 'unhone', 'unka', 'unkaa',
    'unke', 'unki', 'unko', 'unless', 'unlikely', 'unn', 'unse', 'until', 'unto', 'up', 'upar',
    'upon', 'us', 'use', 'used', 'useful', 'uses', 'usi', 'using', 'uska', 'uske', 'usne', 'uss',
    'usse', 'ussi', 'usually', 'vaala', 'vaale', 'vaali', 'vahaan', 'vahan', 'vahi', 'vahin', 'vaisa',
    'vaise', 'vaisi', 'vala', 'vale', 'vali', 'various', 've', 'very', 'via', 'viz', 'vo', 'waala',
    'waale', 'waali', 'wagaira', 'wagairah', 'wagerah', 'waha', 'wahaan', 'wahan', 'wahi', 'wahin',
    'waisa', 'waise', 'waisi', 'wala', 'wale', 'wali', 'want', 'wants', 'was', 'wasn', 'wasnt',
    "wasn't", 'way', 'we', "we'd", 'well', "we'll", 'went', 'were', "we're", 'weren', 'werent',
    "weren't", "we've", 'what', 'whatever', "what's", 'when', 'whence', 'whenever', 'where',
    'whereafter', 'whereas', 'whereby', 'wherein', "where's", 'whereupon', 'wherever', 'whether',
    'which', 'while', 'who', 'whoever', 'whole', 'whom', "who's", 'whose', 'why', 'will', 'willing',
    'with', 'within', 'without', 'wo', 'woh', 'wohi', 'won', 'wont', "won't", 'would', 'wouldn',
    'wouldnt', "wouldn't", 'y', 'ya', 'yadi', 'yah', 'yaha', 'yahaan', 'yahan', 'yahi', 'yahin', 'ye',
    'yeah', 'yeh', 'yehi', 'yes', 'yet', 'you', "you'd", "you'll", 'your', "you're", 'yours',
    'yourself', 'yourselves', "you've", 'yup'
})

In [12]:
hindi_stop = set(["अंदर","अत","अदि","अप","अपना","अपनि","अपनी","अपने","अभि",
                  "अभी","आदि","आप","इंहिं","इंहें","इंहों","इतयादि","इत्यादि","इन",
                  "इनका","इन्हीं","इन्हें","इन्हों","इस","इसका","इसकि","इसकी","इसके",
                  "इसमें","इसि","इसी","इसे","उंहिं","उंहें","उंहों","उन","उनका","उनकि",
                  "उनकी","उनके","उनको","उन्हीं","उन्हें","उन्हों","उस","उसके","उसि",
                  "उसी","उसे","एक","एवं","एस","एसे","ऐसे","ओर","और","कइ","कई",
                  "कर","करता","करते","करना","करने","करें","कहते","कहा","का","काफि",
                  "काफ़ी","कि","किंहें","किंहों","कितना","किन्हें","किन्हों","किया","किर","किस",
                  "किसि","किसी","किसे","की","कुछ","कुल","के","को","कोइ","कोई","कोन",
                  "कोनसा","कौन","कौनसा","गया","घर","जब","जहाँ","जहां","जा","जिंहें","जिंहों",
                  "जितना","जिधर","जिन","जिन्हें","जिन्हों","जिस","जिसे","जीधर","जेसा","जेसे",
                  "जैसा","जैसे","जो","तक","तब","तरह","तिंहें","तिंहों","तिन","तिन्हें","तिन्हों",
                  "तिस","तिसे","तो","था","थि","थी","थे","दबारा","दवारा","दिया","दुसरा","दुसरे",
                  "दूसरे","दो","द्वारा","न","नहिं","नहीं","ना","निचे","निहायत","नीचे","ने","पर",
                  "पहले","पुरा","पूरा","पे","फिर","बनि","बनी","बहि","बही","बहुत","बाद",
                  "बाला","बिलकुल","भि","भितर","भी","भीतर","मगर","मानो","मे","में","यदि",
                  "यह","यहाँ","यहां","यहि","यही","या","यिह","ये","रखें","रवासा","रहा","रहे",
                  "ऱ्वासा","लिए","लिये","लेकिन","व","वगेरह","वरग","वर्ग","वह","वहाँ","वहां",
                  "वहिं","वहीं","वाले","वुह","वे","वग़ैरह","संग","सकता","सकते","सबसे","सभि",
                  "सभी","साथ","साबुत","साभ","सारा","से","सो","हि","ही","हुअ","हुआ","हुइ",
                  "हुई","हुए","हे","हें","है","हैं","हो","होता","होति","होती","होते","होना","होने"])

In [13]:
combined_stopwords = eng_stop.union(hindi_stop).union(hinglish_stop)

In [15]:
from indicnlp.tokenize import indic_tokenize
import nltk
nltk.download('punkt_tab')
import pandas as pd
from nltk.tokenize import word_tokenize

def tokenize_combined(text):
    lang = detect_language(text)
    if lang=='en':
        tokens = word_tokenize(text)
    elif lang == 'hi':
        tokens = indic_tokenize.trivial_tokenize(text, lang='hi')
    # removing stop words
    filtered_tokens = [t for t in tokens if t.lower() not in combined_stopwords]
    return filtered_tokens

df['tokens_combined'] = df['cleaned_message'].apply(tokenize_combined)
df.head()

[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\mhsuh\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


,address,body,readable_date,service_center,date,date_sent,cleaned_message,language,tokens_combined
0,JM-PAYZAP-S,Login attempt on your PayZapp account at 06:52...,15 May 2025 7:32:18 pm,+917021075036,1747317738643,1747317736000,Login attempt on your PayZapp account at 0652 ...,en,"[Login, attempt, PayZapp, account, 0652, PM, c..."
1,51501,971115 Message ID: 0PYGGQiwfmU,15 May 2025 7:48:36 pm,+918299901123,1747318716822,1747318715000,NUM Message ID 0PYGGQiwfmU,en,"[NUM, Message, ID, 0PYGGQiwfmU]"
2,VM-HDFCBK-S,Sent Rs.420.00\nFrom HDFC Bank A/C *9442\nTo S...,15 May 2025 8:51:33 pm,+919823000122,1747322493534,1747322491000,Sent Rs42000 From HDFC Bank AC NUM To SUNY On ...,en,"[Rs42000, HDFC, Bank, AC, NUM, SUNY, 150525, R..."
3,AE-AIRTEL-P,फ्री हैलोट्यून आपका इंतज़ार कर रही है!\nआज ही ए...,16 May 2025 12:37:35 pm,+919831029607,1747379255850,1747379253000,फ्री हैलोट्यून आपका इंतज़ार कर रही है आज ही एयर...,hi,"[फ्री, हैलोट्यून, आपका, इंतज़ार, रही, आज, एयरटे..."
4,AE-AIRSLF,"नमस्ते, अपना करेंट पैक डिटेल घर बैठे एयरटेल थै...",16 May 2025 2:23:40 pm,+919840011991,1747385620009,1747385618000,नमस्ते अपना करेंट पैक डिटेल घर बैठे एयरटेल थैं...,hi,"[नमस्ते, करेंट, पैक, डिटेल, बैठे, एयरटेल, थैंक..."


## Computing Cost

Gradient descent involves repeated steps to adjust the value of your parameter $(w,b)$ to gradually get a smaller and smaller cost $J(w,b)$.
- At each step of gradient descent, it will be helpful for you to monitor your progress by computing the cost $J(w,b)$ as $(w,b)$ gets updated. 
- In this section, you will implement a function to calculate $J(w,b)$ so that you can check the progress of your gradient descent implementation.

#### Cost function
As you may recall from the lecture, for one variable, the cost function for linear regression $J(w,b)$ is defined as

$$J(w,b) = \frac{1}{2m} \sum\limits_{i = 0}^{m-1} (f_{w,b}(x^{(i)}) - y^{(i)})^2$$ 

- You can think of $f_{w,b}(x^{(i)})$ as the model's prediction of your restaurant's profit, as opposed to $y^{(i)}$, which is the actual profit that is recorded in the data.
- $m$ is the number of training examples in the dataset

#### Model prediction

- For linear regression with one variable, the prediction of the model $f_{w,b}$ for an example $x^{(i)}$ is representented as:

$$ f_{w,b}(x^{(i)}) = wx^{(i)} + b$$

This is the equation for a line, with an intercept $b$ and a slope $w$

In [ ]:
def compute_cost(x, y, w, b): 
    """
    Computes the cost function for linear regression.
    
    Args:
        x (ndarray): Shape (m,) Input to the model (Population of cities) 
        y (ndarray): Shape (m,) Label (Actual profits for the cities)
        w, b (scalar): Parameters of the model
    
    Returns
        total_cost (float): The cost of using w,b as the parameters for linear regression
               to fit the data points in x and y
    """
    # number of training examples
    m = x.shape[0] 
    
    # You need to return this variable correctly
    total_cost = 0
    f_wb = w * x + b
    error = f_wb - y
    sq_error = error ** 2
    total_error = np.sum(sq_error)
    total_cost = total_error/(2*m)

    return total_cost

## Gradient descent
The gradient descent algorithm is:

$$\begin{align*}& \text{repeat until convergence:} \; \lbrace \newline \; & \phantom {0000} b := b -  \alpha \frac{\partial J(w,b)}{\partial b} \newline       \; & \phantom {0000} w := w -  \alpha \frac{\partial J(w,b)}{\partial w} \tag{1}  \; & 
\newline & \rbrace\end{align*}$$

where, parameters $w, b$ are both updated simultaniously and where  
$$
\frac{\partial J(w,b)}{\partial b}  = \frac{1}{m} \sum\limits_{i = 0}^{m-1} (f_{w,b}(x^{(i)}) - y^{(i)}) \tag{2}
$$
$$
\frac{\partial J(w,b)}{\partial w}  = \frac{1}{m} \sum\limits_{i = 0}^{m-1} (f_{w,b}(x^{(i)}) -y^{(i)})x^{(i)} \tag{3}
$$
* m is the number of training examples in the dataset

    
*  $f_{w,b}(x^{(i)})$ is the model's prediction, while $y^{(i)}$, is the target value

* `compute_gradient` function calculates $\frac{\partial J(w)}{\partial w}$, $\frac{\partial J(w)}{\partial b}$ 

In [ ]:
def compute_gradient(x, y, w, b): 
    """
    Computes the gradient for linear regression 
    Args:
      x (ndarray): Shape (m,) Input to the model (Population of cities) 
      y (ndarray): Shape (m,) Label (Actual profits for the cities)
      w, b (scalar): Parameters of the model  
    Returns
      dj_dw (scalar): The gradient of the cost w.r.t. the parameters w
      dj_db (scalar): The gradient of the cost w.r.t. the parameter b     
     """
    
    # Number of training examples
    m = x.shape[0]
    
    # You need to return the following variables correctly
    dj_dw = 0
    dj_db = 0
    gradient = ( ( w * x + b ) - y )
    dj_db = np.sum(gradient)/m
    gradient = gradient * x
    dj_dw = np.sum(gradient)/m
    return dj_dw, dj_db

### Learning parameters using batch gradient descent

In [ ]:
def gradient_descent(x, y, w_in, b_in, cost_function, gradient_function, alpha, num_iters): 
    """
    Performs batch gradient descent to learn theta. Updates theta by taking 
    num_iters gradient steps with learning rate alpha
    
    Args:
      x :    (ndarray): Shape (m,)
      y :    (ndarray): Shape (m,)
      w_in, b_in : (scalar) Initial values of parameters of the model
      cost_function: function to compute cost
      gradient_function: function to compute the gradient
      alpha : (float) Learning rate
      num_iters : (int) number of iterations to run gradient descent
    Returns
      w : (ndarray): Shape (1,) Updated values of parameters of the model after
          running gradient descent
      b : (scalar)                Updated value of parameter of the model after
          running gradient descent
    """
    
    # number of training examples
    m = len(x)
    
    # An array to store cost J and w's at each iteration — primarily for graphing later
    J_history = []
    w_history = []
    w = copy.deepcopy(w_in)  #avoid modifying global w within function
    b = b_in
    
    for i in range(num_iters):

        # Calculate the gradient and update the parameters
        dj_dw, dj_db = gradient_function(x, y, w, b )  

        # Update Parameters using w, b, alpha and gradient
        w = w - alpha * dj_dw               
        b = b - alpha * dj_db               

        # Save cost J at each iteration
        if i<100000:      # prevent resource exhaustion 
            cost =  cost_function(x, y, w, b)
            J_history.append(cost)

        # Print cost every at intervals 10 times or as many iterations if < 10
        if i% math.ceil(num_iters/10) == 0:
            w_history.append(w)
            print(f"Iteration {i:4}: Cost {float(J_history[-1]):8.2f}   ")
        
    return w, b, J_history, w_history #return w and J,w history for graphing